<a href="https://colab.research.google.com/github/php4nuke/ru-gpts/blob/master/examples/ruGPT3XL_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install env

In [ ]:
!export CUDA_HOME=/usr/local/cuda-10.1
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
!apt-get install llvm-9-dev
!pip install cpufeature
!pip install triton==0.2.3
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed==0.3.7
!ds_report
# And this cell should be run without errors
import deepspeed.ops.sparse_attention.sparse_attn_op
!git clone https://github.com/php4nuke/ru-gpts.git
!pip install transformers==3.5.1
!pip install natsort

### Load model

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append("ru-gpts/gw")
from generation_wrapper import RuGPT3XL
gpt = RuGPT3XL.from_pretrained("sberbank-ai/rugpt3xl", seq_len=1024)

### Simple generation

In [8]:
def filter_resuls(nr):
    return [x[:x.find("<|endoftext|>")] for x in nr]

import requests
t_in = requests.post('https://gpt3.000webhostapp.com/gpt.php', data = {'txt':''})

while t_in.text != "":
    t_out = filter_resuls(gpt.generate(
        t_in.text,
        min_length=300,
        max_length=500,
        temperature=0.8,
        no_repeat_ngram_size=3,
        repetition_penalty=2.0,
    ))
    t_in = requests.post('https://gpt3.000webhostapp.com/gpt.php', data = {'txt':t_out})